In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
min_loss = 0
max_profit = 0
commission = 5


C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [3]:
symbols = """
select distinct(name) from stockdatamonthly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [5]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        

sql1= """
select distinct(name) from stockdatamonthly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [6]:
from tqdm import tqdm
symbol_list = random.sample(symbol_list, 100)
pbar = tqdm(total= len(symbol_list))

for symbol in symbol_list:
    symbol_data = """
    select * from stockdatamonthly where name = '%s'
    """ % symbol
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)
    pbar.update(1)

 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:05<00:00, 19.90it/s]

In [7]:
sql_dates = """
select distinct(date) from stockdatamonthly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [8]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global min_loss
    global max_profit
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatamonthly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        buy_stocks(current_price, current_symbol, date)   

                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:                        
                        sell_stocks(current_symbol, current_price, date)
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [9]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global risk
    global commission
    
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
    else:
        pass

def sell_stocks(current_symbol, current_price, date):
    global position
    global remaining_cash
    global profit
    global commission
    global min_loss
    global max_profit
    global win
    global loss
    
    current_symbol.stop_order = None
    print("Selling ",position[current_symbol.symbol], "stocks of", current_symbol.symbol, " at price", current_price, "on date", date)
    sold = current_price * position[current_symbol.symbol] - commission
    profit_today = sold - current_symbol.bought * position[current_symbol.symbol]
    if profit_today > 0:
        if current_symbol.flag_for_wins == False:
            win += 1
        current_symbol.flag_for_wins = False

    else:
        loss += 1
        current_symbol.flag_for_wins = False

    profit += profit_today

    if profit < min_loss:
        min_loss = profit
    if profit > max_profit:
        max_profit = profit

    remaining_cash = remaining_cash + sold
    del position[current_symbol.symbol]
    current_symbol.bought = None

In [10]:
automatic_trader(symbol_list)

Buying  7.0 stocks of  CNXN At price 48.88 on date 2000-04-28
Buying  26.0 stocks of  HMNY At price 11.81 on date 2000-04-28
Buying  21.0 stocks of  ASTE At price 30.38 on date 2000-04-28
Buying  136.0 stocks of  MAYS At price 6.125 on date 2000-04-28
Buying  46.0 stocks of  BPT At price 11.44 on date 2000-05-31
Buying  66.0 stocks of  TSBK At price 11.0 on date 2000-06-30
Buying  68.0 stocks of  RRC At price 3.313 on date 2000-06-30
Buying  33.0 stocks of  ESCA At price 18.31 on date 2000-07-31
Selling  21.0 stocks of ASTE  at price 21.25 on date 2000-08-31
Buying  50.0 stocks of  MSN At price 2.938 on date 2000-08-31
Buying  29.0 stocks of  KR At price 23.75 on date 2000-09-29
Buying  11.0 stocks of  FMS At price 30.88 on date 2000-09-29
Buying  2.0 stocks of  NANO At price 63.88 on date 2000-09-29
Buying  8.0 stocks of  IEX At price 36.0 on date 2000-12-29
Selling  26.0 stocks of HMNY  at price 3.5 on date 2000-12-29
Buying  5.0 stocks of  IYF At price 88.56 on date 2000-12-29
Selli

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00, 19.90it/s]

Buying  22.0 stocks of  GJH At price 10.32 on date 2012-04-30
Buying  20.0 stocks of  FBHS At price 25.73 on date 2012-08-31
Selling  19.0 stocks of CATM  at price 24.7 on date 2012-12-31
Selling  5.0 stocks of FMS  at price 35.66 on date 2013-01-31
Buying  28.0 stocks of  IMI At price 9.89 on date 2013-01-31
Buying  56.0 stocks of  CTU At price 27.2 on date 2013-01-31
Buying  7.0 stocks of  KR At price 33.28 on date 2013-03-28
Buying  2.0 stocks of  IEX At price 57.38 on date 2013-05-31
Buying  2.0 stocks of  WBC At price 78.62 on date 2013-05-31
Buying  148.0 stocks of  APHB At price 0.71 on date 2013-07-31
Buying  6.0 stocks of  CATM At price 37.08 on date 2013-08-30
Buying  10.0 stocks of  RESI At price 23.3355 on date 2013-09-30
Selling  56.0 stocks of CTU  at price 24.66 on date 2013-09-30
Selling  4.0 stocks of THD  at price 67.88 on date 2014-01-31
Selling  28.0 stocks of IMI  at price 4.15 on date 2014-02-28
Buying  14.0 stocks of  FFNW At price 11.48 on date 2014-06-30
Sellin

In [11]:
print("Remaining Cash ::" , remaining_cash)

Remaining Cash :: 4369.913000000007


In [12]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  11801.624999999995
Minimum Loss is  -1305.4179999999992
Profit 11775.336899999993
Remaining cash  21250.336900000013
Win Percentage
50.0
